In [1]:
import numpy as np
import re

In [3]:
with open("../data/phenol.tripos.mol2") as file:
    data = file.read()

# Left (Amber) workflow

Tripos MOL2 \rightarrow GAFF MOL2 \rightarrow FRCMOD and PRMTOP

In [30]:
!antechamber -fi mol2 -i ../data/phenol.tripos.mol2 -fo mol2 -o phenol.gaff.mol2


Welcome to antechamber 17.3: molecular input file processor.

acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.




See `tleap.gaff.in`.

Then use `parmchk2`...

# Right (OpenFF) workflow

Tripols MOL2 \rightarrow OEMols \rightarrow OpenMM system \rightarrow MOL2, PRMTOP, FRCMOD (via ParmED)

In [8]:
from openeye.oechem import (
    oemolistream, oemolostream, OEIFlavor_MOL2_Forcefield,
    OEIFlavor_Generic_Default, OEIFlavor_PDB_Default, OEIFlavor_PDB_ALL,
    OEFormat_MOL2, OEFormat_MOL2H, OEWriteMolecule, OETriposAtomNames, OEMol,
    OEFormat_PDB, OESmilesToMol, OEAddExplicitHydrogens, OEHasAtomIdx,
    OEAtomGetResidue)

from openforcefield.typing.engines.smirnoff import (
    ForceField, generateTopologyFromOEMol, generateGraphFromTopology, unit)


## Part 1

In [60]:
molecules = []
ifs = oemolistream()
ifs.SetFlavor(OEFormat_MOL2, OEIFlavor_MOL2_Forcefield)
ifs.open("../data/phenol.tripos.mol2")
for index, mol in enumerate(ifs.GetOEMols()):
    molecules.append(OEMol(mol))

## Part 2

In [66]:
topology = generateTopologyFromOEMol(molecules[0])
ff = ForceField('forcefield/smirnoff99Frosst.offxml')
system = ff.createSystem(
    topology,
    molecules,
    nonbondedCutoff=1.1 * unit.nanometer,
    ewaldErrorTolerance=1e-4)

In [67]:
system.getNumParticles()

13


## Part 3

In [68]:
import parmed as pmd

In [74]:
structure = pmd.openmm.load_topology(topology, system)

In [75]:
structure.save("phenol.prmtop")

In [78]:
pmd.tools.writeFrcmod(pmd.load_file("phenol.prmtop"), "phenol.frcmod").execute()